From: [https://gsarantitis.wordpress.com/2020/04/29/auc-roc-gains-chart-and-lift-curve-explained-with-business-implications/](https://gsarantitis.wordpress.com/2020/04/29/auc-roc-gains-chart-and-lift-curve-explained-with-business-implications/)

Data source: [http://archive.ics.uci.edu/dataset/222/bank+marketing](http://archive.ics.uci.edu/dataset/222/bank+marketing)

In [ ]:
#Import modules
import os
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style("whitegrid")
import matplotlib.pyplot as plt
%matplotlib inline
import scikitplot as skplt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import confusion_matrix, roc_curve, auc, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

import mlflow

In [ ]:
artifact_dir = os.path.join(os.getcwd(),"artifacts")
figures_dir = os.path.join(artifact_dir, "figures")
os.makedirs(figures_dir, exist_ok=True)

In [ ]:
#Import and preprocess data
df = pd.read_csv('data/bank-full.csv', sep=';')
 
df.head()

In [ ]:
df.describe()

In [ ]:
#Encode categorical variables
print('Converting categorical variables to numeric...')
 
categorical_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 
                       'contact', 'day', 'month', 'campaign','previous', 'poutcome']
 
use_cols = ['month', 'campaign', 'y']
 
df['y'] = df['y'].map({'yes':1, 'no':0})
 
for c in categorical_columns:
    df[c] = df[c].astype('category')
    df[c] = df[c].cat.codes

In [ ]:
def calculate_scores(x_test, y_test, y_pred, y_pred_prob, fpr, tpr):

    scores={}
    scores['ROC_AUC'] = auc(fpr, tpr)
    scores['Acc'] = accuracy_score(y_test, y_pred)
    scores['Rec'] = recall_score(y_test, y_pred)
    scores['Prec'] = precision_score(y_test, y_pred)
    scores['F1'] = f1_score(y_test, y_pred)
        
    print(f"Modelname: {modelname}")
    print('AUC: {:.2f}'.format(scores['ROC_AUC']))
    print('Accuracy: {:.2f}'.format(scores['Acc']))
    print('Recall: {:.2f}'.format(scores['Rec']))
    print('Precision: {:.2f}'.format(scores['Prec']))
    print('F1-Score: {:.2f}'.format(scores['F1']))

    # clfreport = classification_report(y_test, pred_rf1)
    # print(clfreport)
    
    return scores

In [ ]:
def create_skill_plots(y_test, pred_prob, fpr, tpr, modelname):
    plt.figure(figsize=(15,7))
    
    #Plot AUC-ROC
    ax1 = plt.subplot(1,3,1)
    plt.title('Receiver Operating Characteristic')
    roc_auc = auc(fpr, tpr)
    ax1.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    ax1.legend(loc = 'lower right')
    ax1.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    
    ax2 = plt.subplot(1,3,2)      
    skplt.metrics.plot_cumulative_gain(y_test, pred_prob, ax=ax2)
    
    ax3 = plt.subplot(1,3,3)      
    skplt.metrics.plot_lift_curve(y_test, pred_prob, ax=ax3)

    figname = os.path.join(figures_dir, f"{modelname}_skillplots.png")
    plt.savefig(figname)
    plt.show()
    print(f'saved figure:{figname}')
    
    return figname

In [ ]:
mlflow.sklearn.autolog(disable=True)
Test_Size = 0.25
Experiment_Name= f"Testsize_{Test_Size}"
Author = 'Onur Kerimoglu'
Purpose = f'Model selection'
mlflow.set_experiment(Experiment_Name)

In [ ]:
#Split in 75% train and 25% test set
train_df, test_df = train_test_split(df, test_size = Test_Size, random_state= 42)
 
#Separate target label
y_train = train_df['y']
y_test = test_df['y']
 
#Drop target label
x_train = train_df.drop(['y'], axis = 1)
x_test = test_df.drop(['y'], axis = 1)

In [ ]:
#Build a logit model for testing
basemodelname = "Logit_test"
params = {
        "penalty": None,
        "class_weight": 'balanced'}
parsuf = '_'.join([key.replace('_','')+str(val).replace('.','') for key,val in params.items()])
modelname=f"{basemodelname}_{parsuf}"
    
with mlflow.start_run(run_name=modelname):
    model = LogisticRegression(
        penalty=params['penalty'],
        class_weight=params['class_weight'],
        max_iter=5000)
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)
    y_pred_prob = model.predict_proba(x_test)
    fpr, tpr, threshold = roc_curve(y_test, y_pred_prob[:,1])

    scores = calculate_scores(x_test, y_test, y_pred, y_pred_prob, fpr, tpr)
    figname = create_skill_plots(y_test, y_pred_prob, fpr, tpr, modelname)
    
    mlflow.set_tags({
        "Author": Author,
        "Purpose": Purpose,
        "Experiment": Experiment_Name,
        "Model": modelname,
        "Loader": "sklearn",
        "Test_Size": Test_Size})
    mlflow.log_params(params)
    
    # log the skill metrics
    mlflow.log_metric('ROC_AUC', scores['ROC_AUC'])
    mlflow.log_metric('Accuracy', scores['Acc'])
    mlflow.log_metric('Recall', scores['Rec'])
    mlflow.log_metric('Preicision', scores['Prec'])
    mlflow.log_metric('F1', scores['F1'])

    # log the model as an artifact to enable later use
    mlflow.sklearn.log_model(model, modelname)
    
    # Log the figure as artifact
    mlflow.log_artifact(f"{figname}")

In [ ]:
#Run a parameter sweep for a logit model
basemodelname = "Logit_parsweep"

params_list = {
        "penalty": [None, 'l2'],
        "class_weight": [None, 'balanced']}

with mlflow.start_run(run_name=basemodelname):
    run_i = 0
    for penalty in params_list['penalty']:
        for class_weight in params_list['class_weight']:
            run_i += 1
            params = {
                "penalty": penalty,
                "class_weight": class_weight}

            parsuf = '_'.join([key.replace('_','')+str(val).replace('.','') for key,val in params.items()])
            modelname=f"{basemodelname}_{parsuf}"
            
            with mlflow.start_run(run_name=modelname, nested=True) as subrun_i:
                model = LogisticRegression(
                    penalty=params['penalty'],
                    class_weight=params['class_weight'],
                    max_iter=5000)
                model.fit(x_train, y_train)

                y_pred = model.predict(x_test)
                y_pred_prob = model.predict_proba(x_test)
                fpr, tpr, threshold = roc_curve(y_test, y_pred_prob[:,1])

                scores = calculate_scores(x_test, y_test, y_pred, y_pred_prob, fpr, tpr)
                figname = create_skill_plots(y_test, y_pred_prob, fpr, tpr, modelname)

                mlflow.set_tags({
                    "Author": Author,
                    "Purpose": Purpose,
                    "Experiment": Experiment_Name,
                    "Model": modelname,
                    "Loader": "sklearn",
                    "Test_Size": Test_Size})
                mlflow.log_params(params)

                # log the skill metrics
                mlflow.log_metric('ROC_AUC', scores['ROC_AUC'])
                mlflow.log_metric('Accuracy', scores['Acc'])
                mlflow.log_metric('Recall', scores['Rec'])
                mlflow.log_metric('Preicision', scores['Prec'])
                mlflow.log_metric('F1', scores['F1'])

                # log the model as an artifact to enable later use
                mlflow.sklearn.log_model(model, modelname)

                # Log the figure as artifact
                mlflow.log_artifact(f"{figname}")

In [ ]:
#Run a parameter sweep for Random Forest model
basemodelname = "RandomForest_parsweep"

params_list = {
    "max_depth": [4, 5],
    "n_estimators": [20, 30]}

with mlflow.start_run(run_name=basemodelname):
    run_i = 0
    for max_depth in params_list['max_depth']:
        for n_estimators in params_list['n_estimators']:
            run_i += 1
            params = {
                "max_depth": max_depth,
                "n_estimators": n_estimators}
            
            parsuf = '_'.join([key.replace('_','')+str(val).replace('.','') for key,val in params.items()])
            modelname=f"{basemodelname}_{parsuf}"
            
            with mlflow.start_run(run_name=modelname, nested=True) as subrun_i:
                
                model = RandomForestClassifier(
                    max_depth=params['max_depth'], 
                    n_estimators = params['n_estimators']
                )
                model.fit(x_train, y_train)

                y_pred = model.predict(x_test)
                y_pred_prob = model.predict_proba(x_test)
                fpr, tpr, threshold = roc_curve(y_test, y_pred_prob[:,1])

                scores = calculate_scores(x_test, y_test, y_pred, y_pred_prob, fpr, tpr)
                figname = create_skill_plots(y_test, y_pred_prob, fpr, tpr, modelname)

                mlflow.set_tags({
                    "Author": Author,
                    "Purpose": Purpose,
                    "Experiment": Experiment_Name,
                    "Model": modelname,
                    "Loader": "sklearn",
                    "Test_Size": Test_Size})
                mlflow.log_params(params)

                # log the skill metrics
                mlflow.log_metric('ROC_AUC', scores['ROC_AUC'])
                mlflow.log_metric('Accuracy', scores['Acc'])
                mlflow.log_metric('Recall', scores['Rec'])
                mlflow.log_metric('Preicision', scores['Prec'])
                mlflow.log_metric('F1', scores['F1'])

                # log the model as an artifact to enable later use
                mlflow.sklearn.log_model(model, modelname)
                
                # Log the figure as artifact
                mlflow.log_artifact(f"{figname}")

In [ ]:
mlflow.sklearn.autolog(disable=True)
Test_Size = 0.15
Experiment_Name= f"Testsize_{Test_Size}"
Author = 'Onur Kerimoglu'
Purpose = f'Model selection'
mlflow.set_experiment(Experiment_Name)

In [ ]:
#Split in 85% train and 15% test set
train_df, test_df = train_test_split(df, test_size = 0.15, random_state= 42)
 
#Separate target label
y_train = train_df['y']
y_test = test_df['y']
 
#Drop target label
x_train = train_df.drop(['y'], axis = 1)
x_test = test_df.drop(['y'], axis = 1)

In [ ]:
# Run a parameter sweep for a logit model
basemodelname = "Logit_parsweep"

params_list = {
        "penalty": [None, 'l2'],
        "class_weight": [None, 'balanced']}

with mlflow.start_run(run_name=basemodelname):
    run_i = 0
    for penalty in params_list['penalty']:
        for class_weight in params_list['class_weight']:
            run_i += 1
            params = {
                "penalty": penalty,
                "class_weight": class_weight}

            parsuf = '_'.join([key.replace('_','')+str(val).replace('.','') for key,val in params.items()])
            modelname=f"{basemodelname}_{parsuf}"
            
            with mlflow.start_run(run_name=modelname, nested=True) as subrun_i:
                model = LogisticRegression(
                    penalty=params['penalty'],
                    class_weight=params['class_weight'],
                    max_iter=5000)
                model.fit(x_train, y_train)

                y_pred = model.predict(x_test)
                y_pred_prob = model.predict_proba(x_test)
                fpr, tpr, threshold = roc_curve(y_test, y_pred_prob[:,1])

                scores = calculate_scores(x_test, y_test, y_pred, y_pred_prob, fpr, tpr)
                figname = create_skill_plots(y_test, y_pred_prob, fpr, tpr, modelname)

                mlflow.set_tags({
                    "Author": Author,
                    "Purpose": Purpose,
                    "Experiment": Experiment_Name,
                    "Model": modelname,
                    "Loader": "sklearn",
                    "Test_Size": Test_Size})
                mlflow.log_params(params)

                # log the skill metrics
                mlflow.log_metric('ROC_AUC', scores['ROC_AUC'])
                mlflow.log_metric('Accuracy', scores['Acc'])
                mlflow.log_metric('Recall', scores['Rec'])
                mlflow.log_metric('Preicision', scores['Prec'])
                mlflow.log_metric('F1', scores['F1'])

                # log the model as an artifact to enable later use
                mlflow.sklearn.log_model(model, modelname)

                # Log the figure as artifact
                mlflow.log_artifact(f"{figname}")

In [ ]:
#Run a parameter sweep for Random Forest model
basemodelname = "RandomForest_parsweep"

params_list = {
    "max_depth": [4, 5],
    "n_estimators": [20, 30]}

with mlflow.start_run(run_name=basemodelname):
    run_i = 0
    for max_depth in params_list['max_depth']:
        for n_estimators in params_list['n_estimators']:
            run_i += 1
            params = {
                "max_depth": max_depth,
                "n_estimators": n_estimators}
            
            parsuf = '_'.join([key.replace('_','')+str(val).replace('.','') for key,val in params.items()])
            modelname=f"{basemodelname}_{parsuf}"
            
            with mlflow.start_run(run_name=modelname, nested=True) as subrun_i:
                
                model = RandomForestClassifier(
                    max_depth=params['max_depth'], 
                    n_estimators = params['n_estimators']
                )
                model.fit(x_train, y_train)

                y_pred = model.predict(x_test)
                y_pred_prob = model.predict_proba(x_test)
                fpr, tpr, threshold = roc_curve(y_test, y_pred_prob[:,1])

                scores = calculate_scores(x_test, y_test, y_pred, y_pred_prob, fpr, tpr)
                figname = create_skill_plots(y_test, y_pred_prob, fpr, tpr, modelname)

                mlflow.set_tags({
                    "Author": Author,
                    "Purpose": Purpose,
                    "Experiment": Experiment_Name,
                    "Model": modelname,
                    "Loader": "sklearn",
                    "Test_Size": Test_Size})
                mlflow.log_params(params)

                # log the skill metrics
                mlflow.log_metric('ROC_AUC', scores['ROC_AUC'])
                mlflow.log_metric('Accuracy', scores['Acc'])
                mlflow.log_metric('Recall', scores['Rec'])
                mlflow.log_metric('Preicision', scores['Prec'])
                mlflow.log_metric('F1', scores['F1'])

                # log the model as an artifact to enable later use
                mlflow.sklearn.log_model(model, modelname)
                
                # Log the figure as artifact
                mlflow.log_artifact(f"{figname}")

In [ ]:
import mlflow
from mlflow.entities import ViewType

all_experiments = [exp.experiment_id for exp in mlflow.search_experiments()]
print(all_experiments)

query = f"metrics.F1 > 0.4"

best_run = mlflow.MlflowClient().search_runs(
    experiment_ids=all_experiments,
    filter_string=query,
    run_view_type=ViewType.ALL,
    max_results=1,
    order_by=["metrics.ROC_AUC DESC"]
)[0]
print(best_run)

In [ ]:
#best_run.run_id
run_id = best_run.info.run_id
stage = best_run.info.lifecycle_stage
run_name = best_run.info.run_name
Author = best_run.data.tags["Author"]
Loader = best_run.data.tags["Loader"]
Test_Size = best_run.data.tags["Test_Size"]
print(f"Author: {Author}\nBest run_id: {best_runid}\nRun_name: {run_name}\nTest Size: {Test_Size}\nLoader: {Loader}")

In [ ]:
if True:
    import mlflow
    import pandas as pd
    import pickle
    import xgboost
    from sklearn.model_selection import train_test_split

    #Import and preprocess data
    df = pd.read_csv('data/bank-full.csv', sep=';')

    #Encode categorical variables
    print('Converting categorical variables to numeric...')

    categorical_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 
                           'contact', 'day', 'month', 'campaign','previous', 'poutcome']

    use_cols = ['month', 'campaign', 'y']

    df['y'] = df['y'].map({'yes':1, 'no':0})

    for c in categorical_columns:
        df[c] = df[c].astype('category')
        df[c] = df[c].cat.codes

    #Split in 75% train and 25% test set
    train_df, test_df = train_test_split(df, test_size = float(Test_Size), random_state= 42)

    #Separate target label
    y_train = train_df['y']
    y_test = test_df['y']

    #Drop target label
    x_train = train_df.drop(['y'], axis = 1)
    x_test = test_df.drop(['y'], axis = 1)
    
    if Loader == 'sklearn':
        model = mlflow.sklearn.load_model(f"runs:/{run_id}/{run_name}")
    else:
        raise(Exception(f'Unknown module loader:{Loader}'))
        
    y_pred = model.predict(x_test)
    y_pred_prob = model.predict_proba(x_test)
    fpr, tpr, threshold = roc_curve(y_test, y_pred_prob[:,1])

    scores = calculate_scores(x_test, y_test, y_pred, y_pred_prob, fpr, tpr)
    # figname = create_skill_plots(y_test, y_pred_prob, fpr, tpr, modelname)